In [1]:
from models_new.nac import NAC
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset

from models_new.nac import NeuralAccumulatorCell
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
import os

In [2]:
def reportLoss(loss, epoch):
    print("epoch {},  \t loss {}".format(epoch, loss))
    
def train(model, optimizer, x_train, y_train, epochs, batch_size):
    
    for epoch in range(epochs):
        
        #print('G is: ',model.G.data)
        #print('g is: ',torch.sigmoid(model.G))
        
        for batch in range(len(x_train) // batch_size):
            
            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size),:]

            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                break
                print("nan detected")
            
            loss.backward()
            optimizer.step()
        
        if loss != loss:
            break
        
        if epoch % 250 == 0: 
            #pass
            reportLoss(loss.data, epoch)
            
        
        model.temperature = epoch * 0.001
            
    return test(model,x_train,y_train)
        
def test(model, x_test, y_test):
    
    model.eval()
    output_test = model(x_test)
    loss = F.mse_loss(output_test, y_test)
    
    return loss

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn.parameter import Parameter


class NeuralAccumulatorCell(nn.Module):
    """A Neural Accumulator (NAC) cell [1].
    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.
    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, in_dim, out_dim, ini):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.initial = ini
        self.W_hat = Parameter(torch.Tensor(out_dim, in_dim))
        self.M_hat = Parameter(torch.Tensor(out_dim, in_dim))
        self.temperature = 0.005

        self.register_parameter('W_hat', self.W_hat)
        self.register_parameter('M_hat', self.M_hat)
        self.register_parameter('bias', None)

        self._reset_params(ini)

    def _reset_params(self,ini):
        if ini =='Kai_uni':
            init.kaiming_uniform_(self.W_hat)
            init.kaiming_uniform_(self.M_hat)

        if ini =='Xav_norm':
            init.xavier_normal_(self.W_hat)
            init.xavier_normal_(self.M_hat)

        if ini =='Kai_norm':
            init.kaiming_normal_(self.W_hat)
            init.kaiming_normal_(self.M_hat)

        if ini =='Zeros':
            init.zeros_(self.W_hat)
            init.zeros_(self.M_hat)

        if ini =='Ones':
            init.ones_(self.W_hat)
            init.ones_(self.M_hat)

    def forward(self, input):
        W = torch.tanh(self.W_hat) * torch.sigmoid(self.M_hat)
        return F.linear(input, W, self.bias)

    def extra_repr(self):
        return 'in_dim={}, out_dim={}'.format(
            self.in_dim, self.out_dim
        )

In [4]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn.parameter import Parameter
import numpy as np

class NeuralArithmeticLogicUnitCell(nn.Module):
    """A Neural Arithmetic Logic Unit (NALU) cell [1].

    Attributes:
        in_dim: size of the input sample.
        out_dim: size of the output sample.

    Sources:
        [1]: https://arxiv.org/abs/1808.00508
    """
    def __init__(self, in_dim, out_dim, ini):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.eps = 1e-10

        self.G = Parameter(torch.Tensor(self.out_dim,self.in_dim))
        #print(self.G)
        self.nac = NeuralAccumulatorCell(in_dim, out_dim, ini)
        self.register_parameter('bias', None)
        self.temperature = 0.005
        
        if ini =='Kai_uni':
            init.kaiming_uniform_(self.G, a=math.sqrt(5))

        if ini =='Xav_norm':
            init.xavier_normal_(self.G)

        if ini =='Kai_norm':
            init.kaiming_normal_(self.G)

        if ini =='Zeros':
            init.zeros_(self.G)

        if ini =='Ones':
            init.ones_(self.G)


    def forward(self, input):
        a = self.nac(self.temperature*input)

        g = torch.sigmoid(F.linear(input, self.G, self.bias))
        #g = torch.sigmoid(self.G)
        #print(g)
        add_sub = g * a
        log_input = torch.log(torch.abs(input) + self.eps)
        m = torch.exp(self.nac(log_input))
        mul_div = (1 - g) * m
        y = add_sub + mul_div
        return y
    
    def extra_repr(self):
        return 'in_dim={}, out_dim={}'.format(
            self.in_dim, self.out_dim
        )

In [5]:
test_per_range = 5
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 1000
batch_size = 1

#values = np.linspace(1,1000,10)
#inits = ['Kai_norm']
values = [[0,1],[0,10],[0,100],[-10,10],[-100,100],[-1000,1000]]

train_loss = np.zeros((len(values),test_per_range))
test_loss = np.zeros((len(values),test_per_range))

for j, val in tqdm(enumerate(values)):
    
    print("Testing range: ",val)
    
    test_min_val = val[0]
    test_max_val = val[1]
    
    min_value = 0
    max_value = 1
    
    for i in range(test_per_range):

        model = NeuralArithmeticLogicUnitCell(in_dim, out_dim, 'Kai_norm')
        optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

        x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = None)

        x_test, y_test, _ = generate_synthetic_selection_dataset(test_min_val, test_max_val,
                                                                    sample_size, set_size, boundaries = boundaries)

        train_loss[j, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

        test_loss[j, i]  = test(model, x_test, y_test)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

Testing range:  [0, 1]
epoch 0,  	 loss 116.44195556640625
epoch 250,  	 loss 0.2151702642440796
epoch 500,  	 loss 0.13974036276340485
epoch 750,  	 loss 0.650138795375824
epoch 0,  	 loss 17.60400390625
epoch 250,  	 loss 0.07584310322999954
epoch 500,  	 loss 0.0015391649212688208
epoch 750,  	 loss 0.12700334191322327
epoch 0,  	 loss 40.7957878112793
epoch 250,  	 loss 2.7070817947387695
epoch 500,  	 loss 0.950836181640625
epoch 750,  	 loss 0.06971385329961777
epoch 0,  	 loss 182.17532348632812
epoch 250,  	 loss 0.035194896161556244
epoch 500,  	 loss 0.37300893664360046
epoch 750,  	 loss 0.37376341223716736
epoch 0,  	 loss 2.183091878890991
epoch 250,  	 loss 0.030947769060730934
epoch 500,  	 loss 0.023782700300216675
epoch 750,  	 loss 0.002782810013741255
Testing range:  [0, 10]
epoch 0,  	 loss 345.4822692871094
epoch 250,  	 loss 3.1837666034698486
epoch 500,  	 loss 1.8403294086456299
epoch 750,  	 loss 1.6760534048080444
epoch 0,  	 loss 350.3602294921875
epoch 250, 

In [6]:
np.savetxt("train_temperature_w.csv", np.mean(train_loss,axis=1), delimiter=',', fmt='%2.2f')
np.savetxt("test_temperature_w.csv", np.mean(test_loss,axis=1), delimiter=',', fmt='%2.2f')

In [7]:
test_loss

array([[5.76521545e+02, 2.88264980e+01, 1.13808670e+02, 2.21286774e+02,
        2.72706532e+00],
       [3.67789844e+04, 3.12936777e+04, 4.61900098e+03, 6.97806787e+03,
        1.41748994e+04],
       [2.93838450e+06, 3.30380475e+06, 5.31210300e+06, 5.46541150e+06,
        3.06754225e+06],
       [1.06228271e+03, 7.25237915e+02, 4.13608627e+01, 6.53541565e+02,
        9.14548035e+02],
       [9.50787266e+04, 3.10303691e+04, 3.73622109e+04, 7.64114609e+04,
        4.02363555e+04],
       [1.27414340e+07, 9.74038100e+06, 1.34249410e+07, 7.07834100e+06,
        2.27169300e+06]])

In [8]:
for i in range(len(test_loss)):
    print(np.mean(test_loss[i]))

188.63411049842836
18768.92607421875
4017449.2
679.3942184448242
56023.824609375
9051358.0
